# Project 3 - Setup Graph DB from SQL DB of BART stations

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Included Modules and Packages

Import of interest here is the graph DB neo4j

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Python code to facilitate loading the Neo4j DB and reviewing results.
Our queries are written in neo4j style (i.e., NoSQL) rather than SQL here.

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [7]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [8]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [9]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [10]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [11]:
cursor = connection.cursor()

# Introduction 

We will now go step by step through the process of creating a graph database in Neo4j for the BART sytem. 

We will use some of the queries in 3.2 to pull the needed data in the right format to create nodes and relationships in our Neo4j graph database

We will use the functions created above to create the nodes and relationships:
* my_neo4j_create_node() - creates a node with label Station
* my_neo4j_create_relationship_one_way() - creates a relationship one way between two stations with a weight
* my_neo4j_create_relationship_two_way() - create relationships two way between two stations with a weight

The way we create it might seem a bit strange at first. However, we want to be able to use the canned Neo4j Graph Data Science algorithms, and this design allows us to do so.

# Wipe Out and Verify Neo4j database

Ensure we're starting clean.

In [12]:
my_neo4j_wipe_out_database()
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Query the list of stations and create the departure and arrival nodes in the graph

Use the query from 3.2.1 "Query the list of stations"

For each station X, create two nodes:
* depart X
* arrive X

Use the function my_neo4j_create_node() defined above

For example, West Oakland:
* my_neo4j_create_node('depart West Oakland')
* my_neo4j_create_node('arrive West Oakland')



In [13]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

# We get all the stations database rows back (50) with the station
# Each station is a node that you can depart or arrive from

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

# Verify the number of nodes and relationships

Since there are 50 stations, we should have 100 Nodes in the graph.
One is labeled departStation the other arriveStation


In [14]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


# For all stations, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with weight 0

For each station X and each line Y that the station serves:
* Create a line node
* Create a relationship from the departure node to the line node with weight 0
* Create a relationship from the line node to the arrival node with weight 0

In [15]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

# The lines database has 114 rows
# Therefore our graph of 100 nodes grows by 114 to total 214
# We add 2 relationships for each of the 114 nodes so we get 228 relationships

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [16]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


# Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight

Query the list of all possible line transfers and the transfer times

For each station X, from line Y, to line Z, create a relationship from Y's line node to Z's line node with the weight set to the transfer time


In [17]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)


In [18]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


# Query the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways

Query the list of all segments between each station and its adjoining stations

For each segment from station X to station Y on line Z, create two relationships:
* From X's line node to Y's line node with travel time
* From Y's line node to X's line node with travel time


In [19]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [20]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# Verify the graph we just built checking shortest paths


In [21]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [22]:
my_neo4j_shortest_path('depart Dublin', 'arrive Antioch')


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


In [23]:
my_neo4j_shortest_path('depart SFO', 'arrive OAK')


--------------------------------
   Total Cost:  3882
   Minutes:  64.7
--------------------------------
depart SFO, 0, 0
yellow SFO, 0, 0
yellow San Bruno, 240, 240
yellow South San Francisco, 240, 480
yellow Colma, 180, 660
yellow Daly City, 240, 900
yellow Balboa Park, 240, 1140
green Balboa Park, 48, 1188
green Glen Park, 120, 1308
green 24th Street Mission, 180, 1488
green 16th Street Mission, 120, 1608
green Civic Center, 180, 1788
green Powell Street, 60, 1848
green Montgomery Street, 120, 1968
green Embarcadero, 60, 2028
green West Oakland, 420, 2448
green Lake Merritt, 360, 2808
green Fruitvale, 300, 3108
green Coliseum, 240, 3348
gray Coliseum, 54, 3402
gray OAK, 480, 3882
arrive OAK, 0, 3882


In [24]:
my_neo4j_shortest_path('depart Antioch', 'arrive Berryessa')


--------------------------------
   Total Cost:  7019
   Minutes:  117.0
--------------------------------
depart Antioch, 0, 0
yellow Antioch, 0, 0
yellow Pittsburg Center, 420, 420
yellow Pittsburg, 600, 1020
yellow North Concord, 360, 1380
yellow Concord, 180, 1560
yellow Pleasant Hill, 360, 1920
yellow Walnut Creek, 120, 2040
yellow Lafayette, 300, 2340
yellow Orinda, 300, 2640
yellow Rockridge, 300, 2940
yellow MacArthur, 240, 3180
orange MacArthur, 59, 3239
orange 19th Street, 180, 3419
orange 12th Street, 120, 3539
orange Lake Merritt, 180, 3719
orange Fruitvale, 300, 4019
orange Coliseum, 240, 4259
orange San Leandro, 240, 4499
orange Bay Fair, 240, 4739
orange Hayward, 240, 4979
orange South Hayward, 240, 5219
orange Union City, 300, 5519
orange Fremont, 300, 5819
orange Warm Springs, 360, 6179
orange Milpitas, 540, 6719
orange Berryessa, 300, 7019
arrive Berryessa, 0, 7019
